<a href="https://colab.research.google.com/github/pavelpryadokhin/Telegram-BOT-ChatGPT/blob/main/%D0%9F%D0%BE%D0%B4%D0%B3%D0%BE%D1%82%D0%BE%D0%B2%D0%BA%D0%B0_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85_%D0%B4%D0%BB%D1%8F_ChatGpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai mwclient mwparserfromhell tiktoken


In [ ]:
import mwclient
import mwparserfromhell
import openai
import pandas as pd
import re
import tiktoken
import warnings

warnings.filterwarnings('ignore')

In [ ]:
CATEGORY_TITLE = "Category:Saint Petersburg"
WIKI_SITE = "en.wikipedia.org"

def titles_from_category(
    category: mwclient.listing.Category,
    max_depth: int
) -> set[str]:
    """Возвращает набор заголовков страниц в данной категории Википедии и ее подкатегориях."""
    titles = set()
    for cm in category.members():
        if type(cm) == mwclient.page.Page:
            titles.add(cm.name)
        elif isinstance(cm, mwclient.listing.Category) and max_depth > 0:
            deeper_titles = titles_from_category(cm, max_depth=max_depth - 1)
            titles.update(deeper_titles)
    return titles

# Инициализация объекта MediaWiki
site = mwclient.Site(WIKI_SITE)
category_page = site.pages[CATEGORY_TITLE]
titles = titles_from_category(category_page, max_depth=1)

print(f"Создано {len(titles)} заголовков статей в категории {CATEGORY_TITLE}.")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Создано 424 заголовков статей в категории Category:Saint Petersburg.


In [ ]:
# Задаем секции, которые будут отброшены при парсинге статей
SECTIONS_TO_IGNORE = [
    "See also",
    "References",
    "External links",
    "Further reading",
    "Footnotes",
    "Bibliography",
    "Sources",
    "Citations",
    "Literature",
    "Footnotes",
    "Notes and references",
    "Photo gallery",
    "Works cited",
    "Photos",
    "Gallery",
    "Notes",
    "References and sources",
    "References and notes",
]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def all_subsections_from_section(
    section: mwparserfromhell.wikicode.Wikicode,
    parent_titles: list[str],
    sections_to_ignore: set[str],
) -> list[tuple[list[str], str]]:
    """
    Возвращает список всех вложенных секций для заданной секции страницы Википедии.
    Каждый подраздел представляет собой кортеж из заголовков и текста секции.
    """

    headings = [str(h) for h in section.filter_headings()]
    title = headings[0]
    if title.strip("=" + " ") in sections_to_ignore:
        return []

    titles = parent_titles + [title]
    full_text = str(section)
    section_text = full_text.split(title)[1]

    if len(headings) == 1:
        return [(titles, section_text)]
    else:
        first_subtitle = headings[1]
        section_text = section_text.split(first_subtitle)[0]
        results = [(titles, section_text)]
        for subsection in section.get_sections(levels=[len(titles) + 1]):
            results.extend(all_subsections_from_section(subsection, titles, sections_to_ignore))
        return results

def all_subsections_from_title(
    title: str,
    sections_to_ignore: set[str] = SECTIONS_TO_IGNORE,
    site_name: str = WIKI_SITE,
) -> list[tuple[list[str], str]]:
    """
    Возвращает список всех вложенных секций для заданного заголовка страницы Википедии.
    Каждый подраздел представляет собой кортеж из заголовков и текста.
    """

    site = mwclient.Site(site_name)
    page = site.pages[title]
    text = page.text()
    parsed_text = mwparserfromhell.parse(text)
    headings = [str(h) for h in parsed_text.filter_headings()]

    if headings:
        summary_text = str(parsed_text).split(headings[0])[0]
    else:
        summary_text = str(parsed_text)

    results = [([title], summary_text)]
    for subsection in parsed_text.get_sections(levels=[2]):
        results.extend(all_subsections_from_section(subsection, [title], sections_to_ignore))
    return results

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Разбивка статей на секции
# придется немного подождать, так как на парсинг 100 статей требуется около минуты
wikipedia_sections = []
for title in titles:
    wikipedia_sections.extend(all_subsections_from_title(title))
print(f"Найдено {len(wikipedia_sections)} секций на {len(titles)} страницах")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Найдено 1971 секций на 424 страницах


In [ ]:
def clean_section(section: tuple[list[str], str]) -> tuple[list[str], str]:
    titles, text = section
    text = re.sub(r"<ref.*?</ref>", "", text)  # Удаление ссылок
    text = text.strip()  # Удаление пробелов
    return (titles, text)

# Применяем функцию очистки ко всем секциям
wikipedia_sections = [clean_section(ws) for ws in wikipedia_sections]

def keep_section(section: tuple[list[str], str]) -> bool:
    """Возвращает True, если раздел должен быть сохранен."""
    titles, text = section
    return len(text) >= 16

original_num_sections = len(wikipedia_sections)
wikipedia_sections = [ws for ws in wikipedia_sections if keep_section(ws)]
print(f"Отфильтровано {original_num_sections - len(wikipedia_sections)} секций, осталось {len(wikipedia_sections)} секций.")

Отфильтровано 111 секций, осталось 1860 секций.


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
for ws in wikipedia_sections[:5]:
    print(ws[0])
    display(ws[1][:50] + "...")
    print()

['Template:SaintPetersburg-stub']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'{{asbox\n| image     = Coat of Arms of Saint Peters...'


['Naval Cadet Corps (Russia)']


'[[image:Naval Institute of SPB.jpg|thumb|right|Nav...'

...

'[[File:Рисунки к статье «Морской корпус». Вклейка ...'


['Naval Cadet Corps (Russia)', '==Post Revolution==']


'The College reopened in 1918 to educate officers f...'


['Demographics of Saint Petersburg']


'{{Infobox place demographics|place=[[Saint Petersb...'

In [ ]:
GPT_MODEL = "gpt-3.5-turbo"  # Для выбора токенизатора

def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Возвращает число токенов в строке."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

def halved_by_delimiter(string: str, delimiter: str = "\n") -> list[str]:
    """Разделяет строку на две части, стараясь сбалансировать токены."""
    chunks = string.split(delimiter)
    if len(chunks) == 1:
        return [string, ""]
    elif len(chunks) == 2:
        return chunks
    else:
        total_tokens = num_tokens(string)
        halfway = total_tokens // 2
        best_diff = halfway
        for i, chunk in enumerate(chunks):
            left = delimiter.join(chunks[:i + 1])
            left_tokens = num_tokens(left)
            diff = abs(halfway - left_tokens)
            if diff >= best_diff:
                break
            best_diff = diff
        left = delimiter.join(chunks[:i])
        right = delimiter.join(chunks[i:])
        return [left, right]

def truncated_string(string: str, model: str, max_tokens: int, print_warning: bool = True) -> str:
    """Обрезает строку до максимально разрешенного числа токенов."""
    encoding = tiktoken.encoding_for_model(model)
    encoded_string = encoding.encode(string)
    truncated_string = encoding.decode(encoded_string[:max_tokens])
    if print_warning and len(encoded_string) > max_tokens:
        print(f"Предупреждение: Строка обрезана с {len(encoded_string)} токенов до {max_tokens} токенов.")
    return truncated_string

def split_strings_from_subsection(
    subsection: tuple[list[str], str],
    max_tokens: int = 1000,
    model: str = GPT_MODEL,
    max_recursion: int = 5,
) -> list[str]:
    """Разделяет секции на список частей, каждая с не более чем max_tokens."""
    titles, text = subsection
    string = "\n\n".join(titles + [text])
    num_tokens_in_string = num_tokens(string)

    if num_tokens_in_string <= max_tokens:
        return [string]
    elif max_recursion == 0:
        return [truncated_string(string, model=model, max_tokens=max_tokens)]
    else:
        titles, text = subsection
        for delimiter in ["\n\n", "\n", ". "]:
            left, right = halved_by_delimiter(text, delimiter=delimiter)
            if left == "" or right == "": continue
            else:
                results = []
                for half in [left, right]:
                    half_subsection = (titles, half)
                    half_strings = split_strings_from_subsection(
                        half_subsection,
                        max_tokens=max_tokens,
                        model=model,
                        max_recursion=max_recursion - 1,)
                    results.extend(half_strings)
                return results
    return [truncated_string(string, model=model, max_tokens=max_tokens)]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Делим секции на части
MAX_TOKENS = 1600
wikipedia_strings = []
for section in wikipedia_sections:
    wikipedia_strings.extend(split_strings_from_subsection(section, max_tokens=MAX_TOKENS))

print(f"{len(wikipedia_sections)} секций Википедии поделены на {len(wikipedia_strings)} строк.")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1860 секций Википедии поделены на 1926 строк.


In [ ]:
print(wikipedia_strings[1])

Naval Cadet Corps (Russia)

[[image:Naval Institute of SPB.jpg|thumb|right|Naval Cadet Corps building in 2014]]
The '''Naval Cadet Corps''' ({{lang-ru|Морской кадетский корпус}}), occasionally translated as the Marine Cadet Corps or the Sea Cadet Corps, is an educational establishment for educating naval officers for commissioning in the [[Russian Navy]] in [[Saint Petersburg]].


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from openai import OpenAI
import os
import getpass

EMBEDDING_MODEL = "text-embedding-ada-002"  # Модель токенизации от OpenAI

os.environ["OPENAI_API_KEY"] = getpass.getpass("Введите OpenAI API Key:")
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

def get_embedding(text, model="text-embedding-ada-002"):
    """Получает эмбеддинг для переданного текста."""
    return client.embeddings.create(input=[text], model=model).data[0].embedding

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Введите OpenAI API Key:··········


In [ ]:
SAVE_PATH = "./Saint Petersburg.csv"

df = pd.DataFrame({"text": wikipedia_strings[:10]})
df['embedding'] = df.text.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))
df.to_csv(SAVE_PATH, index=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
df.head(10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,text,embedding
0,Template:SaintPetersburg-stub\n\n{{asbox\n| im...,"[-0.019218014553189278, 0.0011772229336202145,..."
1,Naval Cadet Corps (Russia)\n\n[[image:Naval In...,"[-0.017364492639899254, 0.002858232008293271, ..."
2,Naval Cadet Corps (Russia)\n\n==History==\n\n[...,"[-0.0005884834681637585, 0.005313960835337639,..."
3,Naval Cadet Corps (Russia)\n\n==Post Revolutio...,"[0.004730003885924816, -0.012998382560908794, ..."
4,Demographics of Saint Petersburg\n\n{{Infobox ...,"[0.02548605389893055, -0.0038593898061662912, ..."
5,Demographics of Saint Petersburg\n\n==Ethnicit...,"[0.01871417462825775, 0.007973752915859222, 0...."
6,Demographics of Saint Petersburg\n\n==Ethnicit...,"[0.01630660891532898, 0.01141198817640543, 0.0..."
7,Demographics of Saint Petersburg\n\n==Ethnicit...,"[0.025462275370955467, 0.009829869493842125, 0..."
8,Demographics of Saint Petersburg\n\n==Ethnicit...,"[0.02320416271686554, 0.0060261948965489864, 0..."
9,Demographics of Saint Petersburg\n\n==Ethnicit...,"[0.018607303500175476, 0.006891593802720308, 0..."


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
